In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("MapsaBigData").getOrCreate()

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
filePath = """./databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-clean.parquet"""

airbnbDF = spark.read.parquet(filePath)

(trainDF, testDF) = airbnbDF.randomSplit([.8, .2], seed=42)

categoricalCols = [field for (field, dataType) in trainDF.dtypes if dataType == "string"]

indexOutputCols = [x + "Index" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")

numericCols = [field for (field, dataType) in trainDF.dtypes if ((dataType == "double") & (field != "price"))]

assemblerInputs = indexOutputCols + numericCols

vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

rf = RandomForestRegressor(labelCol="price", maxBins=40, maxDepth=5, numTrees=100, seed=42)

pipeline = Pipeline(stages=[stringIndexer, vecAssembler, rf])

In [4]:
pip install mlflow

  Using cached mlflow-1.28.0-py3-none-any.whl (17.0 MB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached sqlparse-0.4.2-py3-none-any.whl (42 kB)
  Using cached prometheus_flask_exporter-0.20.3-py3-none-any.whl (18 kB)
  Using cached databricks_cli-0.17.3-py3-none-any.whl
  Using cached Flask-2.2.2-py3-none-any.whl (101 kB)
  Using cached docker-5.0.3-py2.py3-none-any.whl (146 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)
  Using cached Werkzeug-2.2.2-py3-none-any.whl (232 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import mlflow
import mlflow.spark
import pandas as pd

with mlflow.start_run(run_name="linear-regression") as run:
    # Log params: num_trees and max_depth
    mlflow.log_param("num_trees", rf.getNumTrees())
    mlflow.log_param("max_depth", rf.getMaxDepth())
    # Log model
    pipelineModel = pipeline.fit(trainDF)
    mlflow.spark.log_model(pipelineModel, "model")
    # Log metrics: RMSE and R2
    predDF = pipelineModel.transform(testDF)
    regressionEvaluator = RegressionEvaluator(predictionCol="prediction",
    labelCol="price")
    rmse = regressionEvaluator.setMetricName("rmse").evaluate(predDF)
    r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF)
    mlflow.log_metrics({"rmse": rmse, "r2": r2})
    # Log artifact: feature importance scores
    rfModel = pipelineModel.stages[-1]
    pandasDF = (pd.DataFrame(list(zip(vecAssembler.getInputCols(),
    rfModel.featureImportances)),
    columns=["feature", "importance"])
    .sort_values(by="importance", ascending=False))
    # First write to local filesystem, then tell MLflow where to find that file
    pandasDF.to_csv("feature-importance.csv", index=False)
    mlflow.log_artifact("feature-importance.csv")

/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
